### Konstantina Andronikou

## This Notebook implements a comparison between Active and Passive

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Active

In [4]:
#removing warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Someone drove Mary to school")
output

{'verbs': [{'verb': 'drove',
   'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARGM-GOL: to school]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-GOL', 'I-ARGM-GOL']}],
 'words': ['Someone', 'drove', 'Mary', 'to', 'school']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone drove Mary to school"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'drove',
     'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARGM-GOL: to school]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-GOL', 'I-ARGM-GOL']}],
   'words': ['Someone', 'drove', 'Mary', 'to', 'school']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} drove John to school.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Margaret'}, {'first_name': 'Anne'}, {'first_name': 'Alice'}, {'first_name': 'Julie'}, {'first_name': 'Sara'}, {'first_name': 'Amy'}, {'first_name': 'Stephanie'}, {'first_name': 'Carolyn'}, {'first_name': 'Patricia'}, {'first_name': 'Judy'}, {'first_name': 'Anthony'}, {'first_name': 'Greg'}, {'first_name': 'Walter'}, {'first_name': 'Harold'}, {'first_name': 'Jason'}, {'first_name': 'Marilyn'}, {'first_name': 'Sandra'}, {'first_name': 'Colin'}, {'first_name': 'Colin'}, {'first_name': 'Edwin'}, {'first_name': 'Cynthia'}, {'first_name': 'Peter'}, {'first_name': 'Alexander'}, {'first_name': 'Betty'}, {'first_name': 'Jerry'}, {'first_name': 'Peter'}, {'first_name': 'Kelly'}, {'first_name': 'Christine'}, {'first_name': 'Ed'}, {'first_name': 'Sophie'}, {'first_name': 'Angela'}, {'first_name': 'Florence'}, {'first_name': 'Mike'}, {'first_name': 'Edith'}, {'first_name': 'Jeff'}, {'first_name': 'Lisa'}, {'first_name': 'Annie'}, {'first_name': 'Joe'}, {'first_name': 'Steven'}

In [12]:
with open('dataset/Active.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Active.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    4 (0.4%)

Example fails:
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
['Margaret', 'drove', 'John', 'to', 'school', '.'] [ True]
['Anne', 'drove', 'John', 'to', 'school', '.'] [ True]
['Alice', 'drove', 'John', 'to', 'school', '.'] [ True]
['Julie', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sara', 'drove', 'John', 'to', 'school', '.'] [ True]
['Amy', 'drove', 'John', 'to', 'school', '.'] [ True]
['Stephanie', 'drove', 'John', 'to', 'school', '.'] [ True]
['Carolyn', 'drove', 'John', 'to', 'school', '.'] [ True]
['Patricia', 'drove', 'John', 'to', 'school', '.'] [ True]
['Judy', 'drove', 'John', 'to', 'school', '.'] [ True]
['Anthony', 'drove', 'John', 'to', 'school', '.'] [ True]
['Greg', 'drove', 'John', 'to', 'school', '.'] [ True]
['Walter', 'drove', 'John', 't

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Active.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## PASSIVE

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description'] #second verb in this case 

In [16]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description'] #second verb 
        
    return predicate_structure

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
editor = Editor()

# create examples
t = editor.template('{first_name} got driven to school by John.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Charlie'}, {'first_name': 'Carl'}, {'first_name': 'Wendy'}, {'first_name': 'Michael'}, {'first_name': 'Bobby'}, {'first_name': 'Tom'}, {'first_name': 'Harry'}, {'first_name': 'Jay'}, {'first_name': 'Katie'}, {'first_name': 'Edward'}, {'first_name': 'Alice'}, {'first_name': 'Bobby'}, {'first_name': 'Bobby'}, {'first_name': 'Ron'}, {'first_name': 'Edith'}, {'first_name': 'Charlotte'}, {'first_name': 'Harold'}, {'first_name': 'John'}, {'first_name': 'David'}, {'first_name': 'Lawrence'}, {'first_name': 'Larry'}, {'first_name': 'Johnny'}, {'first_name': 'Lauren'}, {'first_name': 'Amanda'}, {'first_name': 'Tim'}, {'first_name': 'Lisa'}, {'first_name': 'Larry'}, {'first_name': 'Larry'}, {'first_name': 'Jeff'}, {'first_name': 'Mary'}, {'first_name': 'Peter'}, {'first_name': 'Kenneth'}, {'first_name': 'Lisa'}, {'first_name': 'Edith'}, {'first_name': 'David'}, {'first_name': 'Don'}, {'first_name': 'Alfred'}, {'first_name': 'Grace'}, {'first_name': 'Billy'}, {'first_name': '

In [20]:
with open('dataset/Passive.txt', 'w') as f:
    print(t.data, file = f)

In [21]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Passive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    103 (10.3%)

Example fails:
Nancy got [V: driven] [ARG2: to school] [ARG0: by John] .
----
Claire got [V: driven] [ARG2: to school] [ARG0: by John] .
----
Sophie got [V: driven] [ARG2: to school] [ARG0: by John] .
----
['Charlie', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Carl', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Wendy', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Michael', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Bobby', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Tom', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Harry', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Jay', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Katie', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Edward', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Alice', 'got', 

In [22]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Passive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 